**Tool Use**

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr
from pushover import push, record_unknown_question, record_user_details
from pydantic import BaseModel

load_dotenv(override=True)
openai = OpenAI()

In [ ]:
push("Hi there!")

In [ ]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

tools

In [ ]:
name = "Nhu Nguyen"

reader = PdfReader("me/NhuNguyen.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/NhuNguyenSummary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

# Generator

system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
    particularly questions related to {name}'s career, background, skills and experience. \
    Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
    You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
    Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
    If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    #messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": message}]
    done = False
    # Loop
    #### Send message and get response
    #### Process the initial response and determine whether to execute a tool call or provide a direct response to the user.
    #### If the response indicates a tool call, execute the tools, grab the results, and send them back to the LLM. and start again
    #### Otherwise, return the chat model's final output.
    # End loop
    while not done:
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)
        print(response)
        finish_reason = response.choices[0].finish_reason
        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

#chat("Who is your favorite musician?", [])

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

# The Model Context Protocol (MCP) - The USB-C of Agentic AI

The Model Context Protocol (MCP) is an **open standard** designed to standardize how Large Language Models (LLMs) connect with and utilize external data sources, tools, and services. It was introduced by Anthropic in November 2024 to solve the core challenge of getting AI agents to effectively interact with the real world.

## The Core Problem MCP Solves

Before MCP, if an LLM needed to perform a task requiring external data (like checking a calendar or querying a database), developers had to create custom, brittle integrations for every single tool and LLM combination. This was inefficient and hindered the development of truly capable AI agents.

MCP acts as a **universal connector**, often compared to a "USB-C port for AI applications," eliminating the need for these custom, one-off integrations.


## What it's not

**A framework for building agents**

**A fundamental change to how agents work**

**A way to code agents**

## What it is

**A protocol, a standard**

**A simple way to integrate tools, resources, prompts**

**A USB-C port for AI applications**

## How MCP Works: Architecture

MCP relies on a client-server architecture using JSON-RPC 2.0 messages to structure communication:

1.  **MCP Host (The AI Application):** This is the environment where the user interacts with the AI (e.g., Claude Desktop, an AI-powered IDE, or a custom chat app).

2.  **MCP Client (The Connector):** Located within the Host, the Client manages the connection and translates the LLM's requests into the standardized MCP format.

3.  **MCP Server (The External Tool):** This is a service that connects to a specific external system (like a GitHub repository, a SQL database, or a local file system). The Server receives the standardized request, executes the action using the external tool, and returns the result to the Client.

